In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import visualization
from math import tan, atan, radians, degrees

In [ ]:
# by https://stackoverflow.com/questions/2891790/how-to-pretty-printing-a-numpy-array-without-scientific-notation-and-with-given
import contextlib

@contextlib.contextmanager
def printoptions(*args, **kwargs):
    original = np.get_printoptions()
    np.set_printoptions(*args, **kwargs)
    try:
        yield
    finally: 
        np.set_printoptions(**original)
        

In [ ]:
def construct_proj_matrix(H=1080, W=1914, fov=50.0, near_clip=1.5):
    # for z coord
    f = near_clip  # the near clip, but f in the book
    n = 10003.815  # the far clip, rounded value of median, after very weird values were discarded
    # x coord
    r = W * n * tan(radians(fov) / 2) / H
    l = -r
    # y coord
    t = n * tan(radians(fov) / 2)
    b = -t
    # x00 = 2*n/(r-l)
    x00 = H / (tan(radians(fov) / 2) * W)
    # x11 = 2*n/(t-b)
    x11 = 1 / tan(radians(fov) / 2)
    return np.array([
        [x00, 0, -(r + l) / (r - l), 0],
        [0, x11, -(t + b) / (t - b), 0],
        [0, 0, -f / (f - n), -f * n / (f - n)],
        [0, 0, -1, 0],
    ])

def calc_matrix_params(proj):
    x22 = proj[2, 2]
    x23 = proj[2, 3]
    n = -x23 / x22
    f = -x23 / (x22 - 1)
    return n, f

In [ ]:
ini_file = "gta-postprocessing.ini"
visualization.multi_page = False
visualization.ini_file = ini_file

conn = visualization.get_connection()
cur = conn.cursor()
cur.execute("""SELECT snapshot_id, imagepath, proj_matrix \
  FROM snapshots \
  WHERE run_id = 6
  ORDER BY snapshot_id DESC \
  """)

results = []
for row in cur:
    res = dict(row)
    res['proj_matrix'] = np.array(res['proj_matrix'])
    results.append(res)

print('There are {} records'.format(len(results)))
projs = [i['proj_matrix'] for i in results]

# todo: estimate far plane by least squares. The rest is known. Before that, get rid of all outliers
# x00 = proj[0, 0]
# x11 = proj[1, 1]
x22s = np.array([proj[2, 2] for proj in projs])
x23s = np.array([proj[2, 3] for proj in projs])

ns = -x23s / x22s
fs = -x23s / (x22s - 1)

In [ ]:
# showing plots with values
%matplotlib inline

fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$X_{22}$')
plt.hist(x22s, bins=1000)

fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$X_{23}$')
plt.hist(x23s, bins=1000)


fig = plt.figure(figsize=(14, 14))
plt.title('both')
plt.xlabel('$X_{22}$')
plt.ylabel('$X_{23}$')
plt.loglog(x22s, x23s, 'o', markersize=1)
plt.show()

In [ ]:
# showing weird matrices
with printoptions(precision=5, suppress=True):
    print('weird x22 value in proj: ')
    weird_projs = [proj for proj in projs if abs(proj[2, 2] - (-1)) < 1e-4]
    for weird_proj in weird_projs:
        print(weird_proj)
        print(calc_matrix_params(weird_proj))
    
    print('correct proj')
    print(projs[0])
    print(calc_matrix_params(projs[0]))

    print('weird x23 value in proj: ')
    weird_projs = [proj for proj in projs if proj[2, 3] > 1000]
    for weird_proj in weird_projs:
        print(weird_proj)
        print(calc_matrix_params(weird_proj))
    
    print('correct proj')
    print(projs[0])
    print(calc_matrix_params(projs[0]))



In [ ]:
# showing plots with n and f
%matplotlib inline
# axes[0].plot(x22s, np.zeros_like(x22s), 'x')
fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$N$')
plt.hist(ns, bins=1000)

fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$F$')
plt.hist(fs, bins=1000)

fig = plt.figure(figsize=(14, 14))
plt.title('both')
plt.xlabel('$N$')
plt.ylabel('F$')
plt.loglog(ns, fs, 'o', markersize=1)
plt.show()

fig = plt.figure(figsize=(14, 14))
plt.title('both')
plt.xlabel('$N$')
plt.ylabel('F$')
plt.plot(ns, fs, 'o', markersize=1)
plt.show()

In [ ]:
# filtered weird values
# this filters only very weird values
correct_projs = [proj for proj in projs if abs(proj[2, 2] - (-1)) > 1e-2 and proj[2, 3] < 1e3]
# this filters bassed on knowledge of near clip (far clip in terminilogy of equations, f)
correct_projs = [proj for proj in correct_projs if abs((-proj[2, 3] / (proj[2, 2] - 1)) - 1.5) < 5e-4]
x22s = np.array([proj[2, 2] for proj in correct_projs])
x23s = np.array([proj[2, 3] for proj in correct_projs])

ns = -x23s / x22s
fs = -x23s / (x22s - 1)

print('all values count: ', len(projs))
print('correct values count: ', len(correct_projs))
print('filtered count: ', len(projs) - len(correct_projs))

In [ ]:
# showing plots with values
%matplotlib inline

fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$X_{22}$')
plt.hist(x22s, bins=1000)

fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$X_{23}$')
plt.hist(x23s, bins=1000)


my_proj = construct_proj_matrix()

fig = plt.figure(figsize=(14, 14))
plt.title('both')
plt.xlabel('$X_{22}$')
plt.ylabel('$X_{23}$')
plt.loglog(x22s, x23s, 'o', markersize=1)
plt.plot(my_proj[2, 2], my_proj[2, 3], 'x', markersize=7)
plt.show()

fig = plt.figure(figsize=(14, 14))
plt.title('both')
plt.xlabel('$X_{22}$')
plt.ylabel('$X_{23}$')
plt.plot(x22s, x23s, 'o', markersize=1)
plt.plot(my_proj[2, 2], my_proj[2, 3], 'x', markersize=7)
plt.show()

dets = [np.linalg.det(proj) for proj in correct_projs]
fig = plt.figure(figsize=(14, 14))
plt.title('determinants')
plt.plot(dets, np.zeros_like(dets), 'o', markersize=1)
plt.plot(np.linalg.det(my_proj), 0, 'x', markersize=7)
plt.show()


In [ ]:
# showing plots with n and f
%matplotlib inline
# axes[0].plot(x22s, np.zeros_like(x22s), 'x')
fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$N$')
plt.hist(ns, bins=3000)

fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$F$')
plt.hist(fs, bins=3000)

fig = plt.figure(figsize=(14, 14))
plt.title('both')
plt.xlabel('$N$')
plt.ylabel('F$')
plt.loglog(ns, fs, 'o', markersize=1)
plt.show()

fig = plt.figure(figsize=(14, 14))
plt.title('both')
plt.xlabel('$N$')
plt.ylabel('$F$')
plt.plot(ns, fs, 'o', markersize=1)
plt.show()

In [ ]:
# fat clip, which I know precisely, is more noisy, near clip is more clear, taking value of the most frequent bin in histogram should suffice
bins = np.linspace(np.min(ns), np.max(ns), num=1000)
#print(bins)
inds = np.digitize(ns, bins)
print(inds)
print(np.sort(np.bincount(inds)))
print(np.argmax(np.bincount(inds)))
print(bins[np.argmax(np.bincount(inds))])
print(bins[np.argmax(np.bincount(inds)) - 1])
print(bins[np.argmax(np.bincount(inds)) + 1])

print('mean: ', np.mean(ns))
print('median: ', np.median(ns))


In [ ]:
# experimenting with generated matrices
my_proj = construct_proj_matrix()
proj = projs[0]
print('my_proj')
print(my_proj)
print('proj')
print(proj)
print('diff')
print(my_proj - proj)
print('diff sum')
print(np.sum(abs(my_proj - proj)))
